In [219]:
import os
import numpy as np

In [220]:
ds_path = "./facebook"

In [221]:
def find_feature_numbers(featfilename, features):
    print("~~~Reading feat file : {}, for features : {}".format(featfilename,features))
    feat2num = dict() # {feature:[no1, no2, ..]}
    featfilepath = os.path.join(ds_path,featfilename)
    with open(featfilepath, "r") as f:
        content = f.readlines()
        for line in content:
            feature = [feature for feature in features if feature in line]
            if feature: feature = feature[0]
            else: continue
            if feature not in feat2num: feat2num[feature] = list()
            
            feature_idx = line.split()[-1]
            feat2num[feature].append(feature_idx)
    return feat2num, len(content)
    

In [216]:
def find_idx(item,list_):
    try:
        return list_.index(item)
    except:
        return -1

def read_features(featfile,feat2num,feat_len):
    featfile = os.path.join(ds_path, featfile)
    featureidxs = feat2num.keys()
    nodedict = dict()
    with open(featfile, "r") as f:
        content = f.read().split()
        start_node = content[0]
        start_idx = 0

        while start_idx < len(content):
              next_idx = start_idx + (feat_len+1)
              node = content[next_idx] 
              features = np.array(content[start_idx+1:next_idx])
              if node not in nodedict: nodedict[node] = dict()  
              for feat_name, feat_idxs in feat2num.keys():
                  nodedict[node][feat_name] = np.array(feat_idxs)
                
                   
              start_idx = next_idx  
              
              

In [217]:
def read_feat_file(featfile, featfilename, features):
    # find feature numbers
    feat2num, feat_len = find_feature_numbers(featfilename,features)
    read_features(featfile, feat2num,feat_len)

In [218]:
def read_feature_files(feat_extract=["gender"]):
    """
    Takes a list of features to extract
    Return in format: {"nodeid": {"feature": feature_value}}
    
    
    # convert binary encoded feature in integer class
    """
    feat_files  = [(file, file.split(".")[0]+".featnames") for file in os.listdir(ds_path) if file.endswith(".feat")]
    feat_files = [("698.feat","698.featnames")]
    for feat_file, feat_file_name in feat_files:
        node_dict = read_feat_file(feat_file, feat_file_name, feat_extract)
        

In [214]:
read_feature_files()

~~~Reading feat file : 698.featnames, for features : ['gender']
857 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
811 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
858 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
859 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '

IndexError: list index out of range